
Débora Buzon da Silva 10851687

# Introdução

Neste projeto estremos analisando e explorando dados do seguinte *dataset* [Hotel Resevations Dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download), com o objetivo de responder à questão: **Será que um hóspede irá cancelar sua reserva?**

Prever se um hóspede irá cancelar sua reserva ou não é de grande importância para os hotéis, pois podem gerar prejuízo. Além disso, a capacidade de prever tais cancelamentos com precisão permite que os hotéis tomem medidas preventivas para evitá-los.

Ao longo desse projeto, iremos implementar métodos para responder à questão proposta usando técnicas básicas como KNN (*K Nearest Neighbours*) e *Naive-Bayes*, e técnicas mais avançadas como *Extra Trees*.

O projeto atualizado pode ser acessado no [Github](https://github.com/dbuzon/classification-challenge).


# Análise Exploratória dos Dados

In [26]:
# importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, ExtraTreesClassifier

from imblearn.over_sampling import RandomOverSampler

In [33]:
# lendo dataframe
df = pd.read_csv('./hotel_reservations.csv')
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


O conjunto possui os seguintes dados:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

## Pré-processamento

Foi feito um mapeamento dos dados não numéricos como *meal_plan*, e *booking_status* para valores correspondentes em números, utilizando a biblioteca LabelEncoder do sklearn.processing. Como por exemplo, no *booking_status* 1 significa que a reserva não foi cancelada e 0 o contrário.

In [4]:
# tratamento dos dados
label_encoder_type_of_meal_plan = LabelEncoder()
label_encoder_room_type_reserved = LabelEncoder()
label_encoder_market_segment_type = LabelEncoder()
label_encoder_booking_status = LabelEncoder()

df['type_of_meal_plan'] = label_encoder_type_of_meal_plan.fit_transform(df['type_of_meal_plan'])
df['room_type_reserved'] = label_encoder_room_type_reserved.fit_transform(df['room_type_reserved'])
df['market_segment_type'] = label_encoder_market_segment_type.fit_transform(df['market_segment_type'])
df['booking_status'] = label_encoder_booking_status.fit_transform(df['booking_status'])

df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,0,0,0,224,2017,10,2,3,0,0,0,65.00,0,1
1,INN00002,2,0,2,3,3,0,0,5,2018,11,6,4,0,0,0,106.68,1,1
2,INN00003,1,0,2,1,0,0,0,1,2018,2,28,4,0,0,0,60.00,0,0
3,INN00004,2,0,0,2,0,0,0,211,2018,5,20,4,0,0,0,100.00,0,0
4,INN00005,2,0,1,1,3,0,0,48,2018,4,11,4,0,0,0,94.50,0,0


Como podemos ver, o conjunto é desbalanceado. O número de instâncias de reservas não canceladas é mais do que o dobro do que o de canceladas.

In [35]:
df['booking_status'].value_counts()

booking_status
Not_Canceled    24390
Canceled        11885
Name: count, dtype: int64

Nos dados que iremos utilizar para predizer Y, iremos remover a coluna *booking_status* (nosso target) e o *Booking_ID*, pois ele não é relevante nas predições. Para tratar o desbalanceamento está sendo utilizada a biblioteca *RandomOverSampler* a qual gera dados fictícios baseados no *dataset* original, de forma a igualar as proporções do valor de *booking_status*.

In [ ]:
X = df.drop(['Booking_ID', 'booking_status'], axis=1)
Y = df['booking_status']

# geração de novos dados para balancear a proporção de reservas canceladas e não canceladas
ros = RandomOverSampler(random_state=0)
X, Y = ros.fit_resample(X, Y)

# Técnicas Utilizadas

Por meio de testes foi determinado que um número maior de folds proporcianava um aumento na acurácia de todos os métodos, alguns com aumentos mais significativos que outros. Por isso estamos utilizando 30 folds.

In [36]:
#definição do cross-validation
kFolds = 30
kf = StratifiedKFold(n_splits=kFolds, shuffle=True)

## KNN

In [58]:
knn = KNeighborsClassifier(n_neighbors=1)
score_knn = cross_val_score(knn, X, Y, cv=kf, n_jobs=-1)
print('Acurácia com 1 K-NN: %0.4f +/- %0.4f' % (score_knn.mean(), score_knn.std()))

Acurácia com 1 K-NN: 0.8997 +/- 0.0071


## Naive-Bayes

O *Naive-Bayes* teve uma acurácia bem baixa utilizando todas as colunas do dataframe, pois nem todos são contínuos. Vemos que quando selecionamos melhor as colunas temos um aumento significativo na acurácia de mais de 10%, porém o KNN continua sendo um método mais adequado para os dados em questão.

In [59]:
nb = GaussianNB()
scores = cross_val_score(nb, X, Y, cv=kf)
print('Acurácia com Naive-Bayes e todas as colunas: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

Acurácia com Naive-Bayes e todas as colunas: 0.5671 +/- 0.0067


In [60]:
Xnb = df[['lead_time', 'avg_price_per_room']]
Ynb = df['booking_status']
ros = RandomOverSampler(random_state=0)
Xnb, Ynb = ros.fit_resample(Xnb, Ynb)
score_nb = cross_val_score(nb, Xnb, Ynb, cv=kf)
print('Acurácia com Naive-Bayes e valores contínuos: %0.4f +/- %0.4f' % (score_nb.mean(), score_nb.std()))

Acurácia com Naive-Bayes e valores contínuos: 0.6814 +/- 0.0096


## Regressão Logística

Similar ao *Naive-Bayes*, a Regressão Logística também não tem uma performance tão boa quanto ao KNN.

In [61]:
rlog = LogisticRegression(max_iter=5000)
score_rlog = cross_val_score(rlog, X, Y, cv=kf, n_jobs=-1)
print('Acurácia com Regressão Logística: %0.4f +/- %0.4f' % (score_rlog.mean(), score_rlog.std()))

Acurácia com Regressão Logística: 0.7789 +/- 0.0093


## SVM

In [ ]:
svm = SVC(kernel='linear')
scores = cross_val_score(svm, X, Y, cv=kf)
print('Acurácia com SVM Linear: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

svm = SVC(kernel='rbf')
scores = cross_val_score(svm, X, Y, cv=kf)
print('Acurácia com SVM RBF: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

svm = SVC(kernel='poly', degree=3)
scores = cross_val_score(svm, X, Y, cv=kf)
print('Acurácia com SVM Poly: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

## Árvores de Decisão

Vemos que o melhor algoritmo para as árvores de decisão é o *entropy*, sendo também o método com maior acurácia até agora.

In [62]:
#classificação
dct = tree.DecisionTreeClassifier()
score_dct_gini = cross_val_score(dct, X, Y, cv=kf)
print('Acurácia com Gini: %0.4f +/- %0.4f' % (score_dct_gini.mean(), score_dct_gini.std())) 

dct = tree.DecisionTreeClassifier(criterion='entropy')
score_dct_entropy = cross_val_score(dct, X, Y, cv=kf)
print('Acurácia com Entropy: %0.4f +/- %0.4f' % (score_dct_entropy.mean(), score_dct_entropy.std())) 

Acurácia com Gini: 0.9292 +/- 0.0052
Acurácia com Entropy: 0.9313 +/- 0.0067


## Ensembles

In [15]:
bag = BaggingClassifier(estimator=SVC(), n_estimators=10)
scores = cross_val_score(bag, X, Y, cv=kf, n_jobs=-1)
print('Acurácia Bagging com SVC: %0.4f +/- %0.4f' % (scores.mean(), scores.std())) 

# Pasting é um Bagging sem reposição de exemplos (bootstrap=False)
bag = BaggingClassifier(estimator=SVC(), n_estimators=10, bootstrap=False)
scores = cross_val_score(bag, X, Y, cv=kf, n_jobs=-1)
print('Acurácia Pasting com SVC: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

/home/derb/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/derb/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/derb/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/derb/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


KeyboardInterrupt: 

In [63]:
rf = RandomForestClassifier()
score_rf = cross_val_score(rf, X, Y, cv=kf, n_jobs=-1)
print('Acurácia Random Forest Gini: %0.4f +/- %0.4f' % (score_rf.mean(), score_rf.std()))

Acurácia Random Forest Gini: 0.9492 +/- 0.0055


In [64]:
bag = AdaBoostClassifier(n_estimators=50)
score_ada = cross_val_score(bag, X, Y, cv=kf, n_jobs=-1)
print('Acurácia Ada Boost: %0.4f +/- %0.4f' % (score_ada.mean(), score_ada.std())) 

Acurácia Ada Boost: 0.8026 +/- 0.0086


### Extra Trees

*Extra Trees* é um algoritmo de aprendizado de máquina que utiliza várias árvores de decisão para fazer previsões sobre dados. Ele é chamado de "Extra" porque, ao contrário de outros algoritmos de árvore de decisão, ele utiliza mais aleatoriedade para criar cada árvore, o que aumenta a robustez do modelo.

O *Extra Trees* é bem similar ao *Random Forest*,aA principal diferença entre eles é que o *Extra Trees* utiliza mais aleatoriedade ao treinar cada árvore, enquanto o *Random Forest* utiliza uma técnica adicional de amostragem de recursos para reduzir a correlação entre as árvores e aumentar a robustez do modelo.

In [56]:
extra_trees = ExtraTreesClassifier()
score_ext = cross_val_score(extra_trees, X, Y, cv=kf, n_jobs=-1)
print('Acurácia Extra Trees Gini: %0.4f +/- %0.4f' % (score_ext.mean(), score_ext.std())) 

Acurácia Extra Trees Gini: 0.9487 +/- 0.0056


# Análise dos Resultados


In [67]:
NaiveBayes = {'Model':'Naive Bayes',
               'Scaling':'Normal Data',
               'Type':'Gaussian',
               'Precision':score_nb.mean()}

DCTGini = {'Model':'Decision Tree',
               'Scaling':'Normal Data',
               'Type': 'Gini',
               'Precision':score_dct_gini.mean()}

DCTEntropy = {'Model':'Decision Tree',
               'Scaling':'Normal Data',
               'Type': 'Gini',
               'Precision':score_dct_entropy.mean()}

Random = {'Model':'Random Forest',
               'Scaling':'Normal Data',
               'Type': 'Gini',
               'Precision':score_rf.mean()}

Extra = {'Model':'Extra Trees',
               'Scaling':'Normal Data',
               'Type': 'Gini',
               'Precision':score_ext.mean()}

KNN = {'Model':'KNN',
               'Scaling':'Normal',
               'Type':'-',
               'Precision':score_knn.mean()}

Logistic = {'Model':'Logistic Regression',
               'Scaling':'Normal',
               'Type':'-',
               'Precision':score_rlog.mean()}

AdaBoosting = {'Model':'AdaBoost',
               'Scaling':'StandardScaler',
               'Type':'-',
               'Precision':score_ada.mean()}

resume = pd.DataFrame({'Naive Bayes':pd.Series(NaiveBayes),
                       'Decision Tree Gini':pd.Series(DCTGini),
                       'Decision Tree Entropy':pd.Series(DCTEntropy),
                       'Random Forest':pd.Series(Random),
                       'Extra Trees':pd.Series(Extra),                       
                       'KNN':pd.Series(KNN),
                       'Logistic Regression':pd.Series(Logistic),
                       'AdaBoost':pd.Series(AdaBoosting),
                      })
resume


,Naive Bayes,Decision Tree Gini,Decision Tree Entropy,Random Forest,Extra Trees,KNN,Logistic Regression,AdaBoost
Model,Naive Bayes,Decision Tree,Decision Tree,Random Forest,Extra Trees,KNN,Logistic Regression,AdaBoost
Scaling,Normal Data,Normal Data,Normal Data,Normal Data,Normal Data,Normal,Normal,StandardScaler
Type,Gaussian,Gini,Gini,Gini,Gini,-,-,-
Precision,0.681406,0.929172,0.931304,0.949221,0.948708,0.899733,0.778905,0.802563


# Conclusão

Vimos que os melhores modelos para o conjunto de dados utilizado foram Árvores de Decisão com o algoritmo *entropy*, *Random Forest* com seus parâmetros padrão e o *Extra Trees*, que é muito similar ao *Random Forest*. Além disso, ao analisar a correlação dos dados, foi possível verificar que o tempo de antecedência (*lead_time*) com que a pessoa faz a reserva e o preço do quarto (*avg_price_per_room*) foram as variáveis mais importantes para a previsão. Isso sugere que as políticas de precificação e gestão de reservas podem ter um impacto significativo na redução de cancelamentos, e, portanto, essas informações podem ser usadas para desenvolver estratégias de negócios mais eficazes.

# Apêndice

## Detecção de água envenenada
Este artigo apresenta uma proposta de detecção de água envenenada através do uso de sinais Wi-Fi e técnicas de machine learning. Quatro classificadores foram utilizados: SVM, k-nearest neighbor, LSTM e Adaboost, sendo avaliados por meio de cross-validation com 5 folds e métricas como AUC, TPR, TNR, F1-Score e Accuracy. Todos os algoritmos apresentaram resultados satisfatórios, com destaque para o KNN com precisão de 82% e Adaboost com 92% na diferenciação entre água envenenada nos níveis de 100 ppm e 1000 ppm.

## Predição da Recidiva de Câncer
O artigo "Um Estudo sobre a Predição da Recidiva de Câncer Usando Técnicas de Aprendizado de Máquina", tinha como objetivo identificar a recidiva de câncer em dados binários, ou seja, identificando se houve ou não tratamento para a doença. Um aspecto interessante deste estudo foi o tratamento dado ao desbalanceamento dos dados, utilizando o SMOTE para aumentar a classe minoritária.

## CatBoost
CatBoost é um gradient boosting que se destaca por lidar de forma mais eficaz com características categóricas em comparação a outras implementações de boosting disponíveis. Desenvolvido pela Yandex, ele introduziu avanços algorítmicos, como o boosting encomendado e um algoritmo inovador para processamento de características categóricas, a fim de combater a mudança de predição causada pelo vazamento de alvo presente em algoritmos de reforço de gradiente existentes. Com desempenho superior em diversos conjuntos de dados, o CatBoost tem se mostrado uma opção promissora para aprimorar a eficácia de algoritmos de machine learning.